In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import roc_curve,roc_auc_score,accuracy_score,f1_score
from scipy.stats import f_oneway
import itertools

In [2]:
datasets = ['H','Y','Q']
tasks = ['D','TL']
sets = ['train','val','test']
num_shuffles=50

pm = u'\u00b1'

In [3]:
h_dict = {
    'acc':{},
    'f1':{},
    'auc':{}
}

y_dict = {
    'acc':{},
    'f1':{},
    'auc':{}
}

q_dict = {
    'acc':{},
    'f1':{},
    'auc':{}
}

avg_dict ={    
    'acc':{},
    'f1':{},
    'auc':{}}

out_dict = {
    'H':h_dict,
    'Y':y_dict,
    'Q':q_dict,
    'avg':avg_dict
}

In [4]:
check=True
for cur_dataset in datasets:
    for cur_task in tasks:
        print('DATASET:',cur_dataset,'\t',cur_task)
        cur_set = 'test'
        true =  pd.read_csv('../../data/output/class_'+cur_dataset+'_'+cur_task+'_'+cur_set+'.csv')
        preds = pd.read_csv('../../data/output/class_'+cur_dataset+'_'+cur_task+'_pred_'+cur_set+'.csv')
        auc = []
        f1 = []
        acc = []
        for i in range(num_shuffles):
            auc.append(roc_auc_score(true.iloc[:,i],preds.iloc[:,i]))
            
            fpr, tpr, thresholds = roc_curve(true.iloc[:,i], preds.iloc[:,i])
            # get the best threshold
            J = tpr - fpr
            ix = np.argmax(J)
            best_thresh = thresholds[ix]
            # print('Best Threshold=%f' % (best_thresh))
            
            y_pred_binned = np.asarray(preds.iloc[:,i] > best_thresh, dtype=int)
            f1.append(f1_score(true.iloc[:,i],y_pred_binned))
            acc.append(accuracy_score(true.iloc[:,i],y_pred_binned))

        print('acc:',np.mean(acc),pm,np.std(acc))
        print('F1:',np.mean(f1),pm,np.std(f1))
        print('auc:',np.mean(auc),pm,np.std(auc))

        out_dict[cur_dataset]['acc'][cur_task] = acc
        out_dict[cur_dataset]['f1'][cur_task] = f1
        out_dict[cur_dataset]['auc'][cur_task] = auc
        if check:
            out_dict['avg']['acc'][cur_task] = list(acc)
            out_dict['avg']['f1'][cur_task] = list(f1)
            out_dict['avg']['auc'][cur_task] = list(auc)
            
        else:
            out_dict['avg']['acc'][cur_task].extend(acc)
            out_dict['avg']['f1'][cur_task].extend(f1)
            out_dict['avg']['auc'][cur_task].extend(auc)
    check = False
    print('\n')

DATASET: H 	 D
acc: 0.851840490797546 ± 0.03865152398511716
F1: 0.9019796580374362 ± 0.028058486791476735
auc: 0.9056213410586035 ± 0.030442877159569785
DATASET: H 	 TL
acc: 0.8839877300613496 ± 0.030240420700598663
F1: 0.9250174424031242 ± 0.021127354129770078
auc: 0.9171500995835596 ± 0.0267512091117952


DATASET: Y 	 D
acc: 0.737258064516129 ± 0.1151966377486566
F1: 0.7563137796831486 ± 0.14507236526224965
auc: 0.7599891156462586 ± 0.1364723373052335
DATASET: Y 	 TL
acc: 0.7403225806451611 ± 0.1111265742506383
F1: 0.7651053444534438 ± 0.13863485216342497
auc: 0.7661986394557823 ± 0.13309882795731


DATASET: Q 	 D
acc: 0.8575 ± 0.06526654749733757
F1: 0.8559778910698891 ± 0.07249891243482329
auc: 0.909078260869565 ± 0.06265026639222704
DATASET: Q 	 TL
acc: 0.8862500000000001 ± 0.05910354614290641
F1: 0.8897913123872633 ± 0.053837823237398046
auc: 0.9339478260869565 ± 0.056977151259039474




In [5]:
datasets = ['HMP_2019_ibdmdb','YachidaS_2019','Qinn_2014']
tasks = ['xgb','xtra']
sets = ['test']
num_shuffles=50
check=True
for cur_dataset in datasets:
    for cur_task in tasks:
        print('DATASET:',cur_dataset,'\t',cur_task)
        filename = '../../data/output/tree-based/classification/'+cur_dataset+'/'+cur_task+'/test.csv'
        test_set = pd.read_csv(filename)

        auc = []
        f1 = []
        acc = []

        for i in range(num_shuffles):
            pred = test_set['Shuffle_'+str(i+1)+'_pred']
            true =  test_set['Shuffle_'+str(i+1)+'_test']
            
            auc.append(roc_auc_score(true,pred))
            
            fpr, tpr, thresholds = roc_curve(true, pred)
            # get the best threshold
            J = tpr - fpr
            ix = np.argmax(J)
            best_thresh = thresholds[ix]
            # print('Best Threshold=%f' % (best_thresh))
            
            y_pred_binned = np.asarray(pred > best_thresh, dtype=int)
            f1.append(f1_score(true,y_pred_binned))
            acc.append(accuracy_score(true,y_pred_binned))
        print('acc:',np.mean(acc),pm,np.std(acc))
        print('F1:',np.mean(f1),pm,np.std(f1))
        print('auc:',np.mean(auc),pm,np.std(auc))

        out_dict[cur_dataset[0]]['acc'][cur_task] = acc
        out_dict[cur_dataset[0]]['f1'][cur_task] = f1
        out_dict[cur_dataset[0]]['auc'][cur_task] = auc
        if check:
            out_dict['avg']['acc'][cur_task] = list(acc)
            out_dict['avg']['f1'][cur_task] = list(f1)
            out_dict['avg']['auc'][cur_task] = list(auc)
            
        else:
            out_dict['avg']['acc'][cur_task].extend(acc)
            out_dict['avg']['f1'][cur_task].extend(f1)
            out_dict['avg']['auc'][cur_task].extend(auc)
    check = False
    print('\n')

DATASET: HMP_2019_ibdmdb 	 xgb
acc: 0.7742944785276075 ± 0.04984861723251688
F1: 0.8450749725626372 ± 0.04240285277894101
auc: 0.8286698050576378 ± 0.034054875236907964
DATASET: HMP_2019_ibdmdb 	 xtra
acc: 0.7471779141104293 ± 0.05159611329351152
F1: 0.8221610610126773 ± 0.04460274620413162
auc: 0.8319258856901441 ± 0.03323361907562904


DATASET: YachidaS_2019 	 xgb
acc: 0.5764516129032258 ± 0.05347546531091172
F1: 0.6054686108401411 ± 0.13505171310276315
auc: 0.5481904761904761 ± 0.04448223114866596
DATASET: YachidaS_2019 	 xtra
acc: 0.5651612903225807 ± 0.04897067439899733
F1: 0.5536037969949665 ± 0.12952948004493553
auc: 0.562174149659864 ± 0.04354676336709887


DATASET: Qinn_2014 	 xgb
acc: 0.8854166666666667 ± 0.034673577035610914
F1: 0.885141887892286 ± 0.03610667553641189
auc: 0.9475478260869565 ± 0.02736213215362499
DATASET: Qinn_2014 	 xtra
acc: 0.8679166666666667 ± 0.0507940421921924
F1: 0.8646472582562322 ± 0.058093250693318374
auc: 0.9307304347826088 ± 0.038890320302518905


In [6]:

for dataset in out_dict.keys():
    print(dataset)
    dataset_dict = out_dict[dataset]
    for metric in dataset_dict.keys():
        print(metric)
        cur_dict = dataset_dict[metric]
        means = {}
        for key in cur_dict.keys():
            means[key] = np.mean(cur_dict[key])
        means = pd.Series(means,index=means.keys())
        best = means.index[np.argmax(means)]

        list_comb = list(itertools.combinations(cur_dict, 2))
        for comb in list_comb:
            if best in comb:
                set0 = cur_dict[comb[0]]
                set1 = cur_dict[comb[1]]

                _,pval = f_oneway(set0,set1)

                print(comb,pval,comb[0],np.mean(set0),pm,np.std(set0),comb[1],np.mean(set1),pm,np.std(set1))

H
acc
('D', 'TL') 1.3397180709136862e-05 D 0.851840490797546 ± 0.03865152398511716 TL 0.8839877300613496 ± 0.030240420700598663
('TL', 'xgb') 2.0885856814722348e-23 TL 0.8839877300613496 ± 0.030240420700598663 xgb 0.7742944785276075 ± 0.04984861723251688
('TL', 'xtra') 4.144073524747149e-29 TL 0.8839877300613496 ± 0.030240420700598663 xtra 0.7471779141104293 ± 0.05159611329351152
f1
('D', 'TL') 1.3083862090014108e-05 D 0.9019796580374362 ± 0.028058486791476735 TL 0.9250174424031242 ± 0.021127354129770078
('TL', 'xgb') 1.506313165750582e-20 TL 0.9250174424031242 ± 0.021127354129770078 xgb 0.8450749725626372 ± 0.04240285277894101
('TL', 'xtra') 2.6442159859454055e-26 TL 0.9250174424031242 ± 0.021127354129770078 xtra 0.8221610610126773 ± 0.04460274620413162
auc
('D', 'TL') 0.04922906525731324 D 0.9056213410586035 ± 0.030442877159569785 TL 0.9171500995835596 ± 0.0267512091117952
('TL', 'xgb') 9.97752124921788e-26 TL 0.9171500995835596 ± 0.0267512091117952 xgb 0.8286698050576378 ± 0.0340548

In [7]:
h_dict = {
    'acc':{},
    'f1':{},
    'auc':{}
}

y_dict = {
    'acc':{},
    'f1':{},
    'auc':{}
}

q_dict = {
    'acc':{},
    'f1':{},
    'auc':{}
}

avg_dict ={    
    'acc':{},
    'f1':{},
    'auc':{}}

out_dict = {
    'H':h_dict,
    'Y':y_dict,
    'Q':q_dict,
    'avg':avg_dict
}

In [11]:
datasets = ['H','Y','Q']
# datasets = ['Y','Q']
tasks = ['head','first','all']
sets = ['test']
num_shuffles=50

pm = u'\u00b1'

In [12]:
check=True
for cur_dataset in datasets:
    for cur_task in tasks:
        print('DATASET:',cur_dataset,'\t',cur_task)
        cur_set = 'test'
        true =  pd.read_csv('../../data/output/class_'+cur_dataset+'_'+'TL'+'_'+cur_task+cur_set+'.csv')
        preds = pd.read_csv('../../data/output/class_'+cur_dataset+'_'+'TL'+'_'+cur_task+'pred_'+cur_set+'.csv')
        auc = []
        f1 = []
        acc = []
        for i in range(num_shuffles):
            auc.append(roc_auc_score(true.iloc[:,i],preds.iloc[:,i]))
            
            fpr, tpr, thresholds = roc_curve(true.iloc[:,i], preds.iloc[:,i])
            # get the best threshold
            J = tpr - fpr
            ix = np.argmax(J)
            best_thresh = thresholds[ix]
            # print('Best Threshold=%f' % (best_thresh))
            
            y_pred_binned = np.asarray(preds.iloc[:,i] > best_thresh, dtype=int)
            f1.append(f1_score(true.iloc[:,i],y_pred_binned))
            acc.append(accuracy_score(true.iloc[:,i],y_pred_binned))

        print('acc:',np.mean(acc),pm,np.std(acc))
        print('F1:',np.mean(f1),pm,np.std(f1))
        print('auc:',np.mean(auc),pm,np.std(auc))

        out_dict[cur_dataset]['acc'][cur_task] = acc
        out_dict[cur_dataset]['f1'][cur_task] = f1
        out_dict[cur_dataset]['auc'][cur_task] = auc
        if check:
            out_dict['avg']['acc'][cur_task] = list(acc)
            out_dict['avg']['f1'][cur_task] = list(f1)
            out_dict['avg']['auc'][cur_task] = list(auc)
            
        else:
            out_dict['avg']['acc'][cur_task].extend(acc)
            out_dict['avg']['f1'][cur_task].extend(f1)
            out_dict['avg']['auc'][cur_task].extend(auc)
    check = False
    print('\n')

DATASET: H 	 head
acc: 0.6391411042944786 ± 0.08262589806137674
F1: 0.7313820090065398 ± 0.08508308673124153
auc: 0.681879413362303 ± 0.08102797485201023
DATASET: H 	 first
acc: 0.8556441717791411 ± 0.041074112167561454
F1: 0.9054348283405892 ± 0.029987099940149944
auc: 0.8880318667390911 ± 0.029901304737890395
DATASET: H 	 all
acc: 0.8839877300613496 ± 0.030240420700598663
F1: 0.9250174424031242 ± 0.021127354129770078
auc: 0.9171500995835596 ± 0.0267512091117952


DATASET: Y 	 head
acc: 0.5445161290322581 ± 0.07377582265434024
F1: 0.5214582099376439 ± 0.21230757432294667
auc: 0.5158367346938776 ± 0.06863150915077712
DATASET: Y 	 first
acc: 0.7206451612903225 ± 0.12702839162516452
F1: 0.7466359619500337 ± 0.142364693217937
auc: 0.7375238095238096 ± 0.16387598263129552
DATASET: Y 	 all
acc: 0.7403225806451611 ± 0.1111265742506383
F1: 0.7651053444534438 ± 0.13863485216342497
auc: 0.7661986394557823 ± 0.13309882795731


DATASET: Q 	 head
acc: 0.6841666666666667 ± 0.07942501704962569
F1: 0

In [13]:
for dataset in out_dict.keys():
    print(dataset)
    dataset_dict = out_dict[dataset]
    for metric in dataset_dict.keys():
        print(metric)
        cur_dict = dataset_dict[metric]
        means = {}
        for key in cur_dict.keys():
            means[key] = np.mean(cur_dict[key])
        means = pd.Series(means,index=means.keys())
        best = means.index[np.argmax(means)]

        list_comb = list(itertools.combinations(cur_dict, 2))
        for comb in list_comb:
            if best in comb:
                set0 = cur_dict[comb[0]]
                set1 = cur_dict[comb[1]]

                _,pval = f_oneway(set0,set1)

                print(comb,pval,comb[0],np.mean(set0),pm,np.std(set0),comb[1],np.mean(set1),pm,np.std(set1))

H
acc
('head', 'all') 1.8049643378188276e-35 head 0.6391411042944786 ± 0.08262589806137674 all 0.8839877300613496 ± 0.030240420700598663
('first', 'all') 0.00018280449712436677 first 0.8556441717791411 ± 0.041074112167561454 all 0.8839877300613496 ± 0.030240420700598663
f1
('head', 'all') 4.9083796668536e-28 head 0.7313820090065398 ± 0.08508308673124153 all 0.9250174424031242 ± 0.021127354129770078
('first', 'all') 0.0003132794071865446 first 0.9054348283405892 ± 0.029987099940149944 all 0.9250174424031242 ± 0.021127354129770078
auc
('head', 'all') 3.7090860956565803e-35 head 0.681879413362303 ± 0.08102797485201023 all 0.9171500995835596 ± 0.0267512091117952
('first', 'all') 1.8017053293337782e-06 first 0.8880318667390911 ± 0.029901304737890395 all 0.9171500995835596 ± 0.0267512091117952
Y
acc
('head', 'all') 3.053391463263096e-17 head 0.5445161290322581 ± 0.07377582265434024 all 0.7403225806451611 ± 0.1111265742506383
('first', 'all') 0.41640915597952555 first 0.7206451612903225 ± 0.1